In [1]:
# Importando bibliotecas
import pandas as pd
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import numpy as np
import networkx as nx
!pip install plotly.express
from plotly import graph_objs as go

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [4]:
# remoção de pontuacao e stopwords

def remove_stopwords(text,lang,domain_stopwords=[]):
  
  stop_words = nltk.corpus.stopwords.words(lang) # lang='portuguese' or lang='english'
  
  s = str(text).lower() # tudo para caixa baixa
  table = str.maketrans({key: None for key in string.punctuation})
  s = s.translate(table) # remove pontuacao
  tokens = word_tokenize(s) #obtem tokens
  v = [i for i in tokens if not i in stop_words and not i in domain_stopwords and not i.isdigit()] # remove stopwords
  s = ""
  for token in v:
    s += token+" "
  return s.strip()


# exemplos de uso
text = "O estudante de Inteligência Artificial foi na livraria comprar  livros para estudar."
text2 = remove_stopwords(text, 'portuguese')
print('Antes: '+text)
print('Depois: '+text2)

Antes: O estudante de Inteligência Artificial foi na livraria comprar  livros para estudar.
Depois: estudante inteligência artificial livraria comprar livros estudar


In [5]:
# stemming
def stemming(text,lang):
  
  stemmer = PorterStemmer() # stemming para ingles
  
  if lang=='portuguese':
    stemmer = nltk.stem.RSLPStemmer() # stemming para portuguese
    
  tokens = word_tokenize(text) #obtem tokens
  
  sentence_stem = ''
  doc_text_stems = [stemmer.stem(i) for i in tokens]
  for stem in doc_text_stems:
    sentence_stem += stem+" "
    
  return sentence_stem.strip()


# exemplos de uso
text = "O estudante de Inteligência Artificial foi na livraria comprar livros para estudar."
text2 = remove_stopwords(text, 'portuguese')
text3 = stemming(text2, 'portuguese')
print('Antes: '+text)
print('Depois: '+text3)

Antes: O estudante de Inteligência Artificial foi na livraria comprar livros para estudar.
Depois: estud intelig artific livr compr livr estud


In [16]:
# obtendo dataset com uma amostra eventos

query = "febre amarela" # query para consultar na base de eventos (max. de 500 respostas)

pd.set_option('display.max_colwidth', -1)
dataset = pd.read_csv('http://websensors.net.br/minicurso/2019/eventos-br-2017.php?q='+query,sep='\t', lineterminator='\n')
dataset

,event_id,date,title,local,url,lat,lng
0,b4b29e9b5537c1896129029bd976438c,2017-01-05 15:27:28,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Ribeirao Preto (SP),http://g1.globo.com/sp/ribeirao-preto-franca/noticia/2017/01/apos-2-morte-por-febre-amarela-em-sp-vacinacao-e-intensificada.html,-21.17,-47.81
1,125bf6fbeef15b17a1d4837127129bbe,2017-01-08 16:04:51,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Ribeirao Preto (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-21.17,-47.81
2,aff14d48f09878b31e0575971fc8fec2,2017-01-08 14:04:51,G1 - Casos de febre amarela em macacos na região preocupam ... - Globo.com,Sao Jose do Rio Preto (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-20.82,-49.37
3,ca7ca58720fc9a2338894095ba6409f0,2017-01-05 13:27:28,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Sao Jose do Rio Preto (SP),http://g1.globo.com/sp/ribeirao-preto-franca/noticia/2017/01/apos-2-morte-por-febre-amarela-em-sp-vacinacao-e-intensificada.html,-20.82,-49.37
4,2a6e78e1226fc9c5f1c9061febf175d1,2017-01-09 14:57:25,Mortes por febre amarela põem em alerta cidades do norte paulista - Istoe,Sao Jose do Rio Preto (SP),http://istoe.com.br/mortes-por-febre-amarela-poem-em-alerta-cidades-do-norte-paulista/,-20.82,-49.37
5,13209ded9adca382ca16e8902477f1d7,2017-01-08 16:04:51,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Aracatuba (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-21.20,-50.43
6,8c6fc6c73c693dcbe885c4dddd2a81a9,2017-01-09 17:22:34,Reunião em Teófilo Otoni discute ações de prevenção à febre ... - Globo.com,Teofilo Otoni (MG),http://g1.globo.com/mg/vales-mg/noticia/2017/01/reuniao-em-teofilo-otoni-discute-acoes-de-prevencao-febre-amarela.html,-17.85,-41.50
7,d53570e38d4e78c9abcf1a475d67308c,2017-01-08 16:04:51,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Catanduva (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-21.13,-48.97
8,281dd88cd8d8898eeb7ca44606265e13,2017-01-09 19:34:32,"Surto de Febre Amarela atinge cidade de Caratinga, no interior de ... - Blasting News",Coronel Fabriciano (MG),http://br.blastingnews.com/brasil/2017/01/surto-de-febre-amarela-atinge-cidade-de-caratinga-no-interior-de-minas-gerais-001381887.html,-19.51,-42.62
9,522f2527c2793d8742a0376929fbec2e,2017-01-09 20:02:08,Governo de Minas investiga casos suspeitos de febre amarela e ... - EBC,Uba (MG),http://agenciabrasil.ebc.com.br/geral/noticia/2017-01/governo-de-minas-investiga-casos-suspeitos-de-febre-amarela-e-malaria-no,-21.12,-42.94


In [17]:
# filtrando apenas o texto do evento
#dataset = dataset[['title']]
dataset

,event_id,date,title,local,url,lat,lng
0,b4b29e9b5537c1896129029bd976438c,2017-01-05 15:27:28,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Ribeirao Preto (SP),http://g1.globo.com/sp/ribeirao-preto-franca/noticia/2017/01/apos-2-morte-por-febre-amarela-em-sp-vacinacao-e-intensificada.html,-21.17,-47.81
1,125bf6fbeef15b17a1d4837127129bbe,2017-01-08 16:04:51,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Ribeirao Preto (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-21.17,-47.81
2,aff14d48f09878b31e0575971fc8fec2,2017-01-08 14:04:51,G1 - Casos de febre amarela em macacos na região preocupam ... - Globo.com,Sao Jose do Rio Preto (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-20.82,-49.37
3,ca7ca58720fc9a2338894095ba6409f0,2017-01-05 13:27:28,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Sao Jose do Rio Preto (SP),http://g1.globo.com/sp/ribeirao-preto-franca/noticia/2017/01/apos-2-morte-por-febre-amarela-em-sp-vacinacao-e-intensificada.html,-20.82,-49.37
4,2a6e78e1226fc9c5f1c9061febf175d1,2017-01-09 14:57:25,Mortes por febre amarela põem em alerta cidades do norte paulista - Istoe,Sao Jose do Rio Preto (SP),http://istoe.com.br/mortes-por-febre-amarela-poem-em-alerta-cidades-do-norte-paulista/,-20.82,-49.37
5,13209ded9adca382ca16e8902477f1d7,2017-01-08 16:04:51,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Aracatuba (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-21.20,-50.43
6,8c6fc6c73c693dcbe885c4dddd2a81a9,2017-01-09 17:22:34,Reunião em Teófilo Otoni discute ações de prevenção à febre ... - Globo.com,Teofilo Otoni (MG),http://g1.globo.com/mg/vales-mg/noticia/2017/01/reuniao-em-teofilo-otoni-discute-acoes-de-prevencao-febre-amarela.html,-17.85,-41.50
7,d53570e38d4e78c9abcf1a475d67308c,2017-01-08 16:04:51,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Catanduva (SP),http://g1.globo.com/sao-paulo/sao-jose-do-rio-preto-aracatuba/alerta-dengue/noticia/2017/01/avanco-da-febre-amarela-na-regiao-preocupa-moradores-e-autoridades.html,-21.13,-48.97
8,281dd88cd8d8898eeb7ca44606265e13,2017-01-09 19:34:32,"Surto de Febre Amarela atinge cidade de Caratinga, no interior de ... - Blasting News",Coronel Fabriciano (MG),http://br.blastingnews.com/brasil/2017/01/surto-de-febre-amarela-atinge-cidade-de-caratinga-no-interior-de-minas-gerais-001381887.html,-19.51,-42.62
9,522f2527c2793d8742a0376929fbec2e,2017-01-09 20:02:08,Governo de Minas investiga casos suspeitos de febre amarela e ... - EBC,Uba (MG),http://agenciabrasil.ebc.com.br/geral/noticia/2017-01/governo-de-minas-investiga-casos-suspeitos-de-febre-amarela-e-malaria-no,-21.12,-42.94


In [18]:
# obtendo a bag-of-words
def compute_bag_of_words(dataset,lang,domain_stopwords=[]):
  
  d = []
  for index,row in dataset.iterrows():
    text = row['title'] #texto do evento
    text2 = remove_stopwords(text, lang,domain_stopwords)
    text3 = stemming(text2, lang)
    d.append(text3)
  
  matrix = CountVectorizer(max_features=1000)
  X = matrix.fit_transform(d)
  
  count_vect_df = pd.DataFrame(X.todense(), columns=matrix.get_feature_names())

  return count_vect_df


bow = compute_bag_of_words(dataset,'portuguese')
bow

,2ª,acab,acontec,adot,afet,afirm,aft,al,alago,alarm,alert,algum,almg,amarel,ampl,ano,anunc,apo,apont,apur,após,aqu,aquidau,ar,aracaju,arax,assinat,atend,atenç,aterror,atest,ating,atrás,aument,açõ,bah,barbacen,barr,bh,biólog,...,sob,sobr,solicit,som,sp,sup,surt,suspeit,tabapuã,temp,ter,teófil,to,tod,tragéd,tranquiliz,tribun,tripl,troc,trê,uberab,un,unidad,uol,urban,vacin,vai,val,valad,vej,vint,vir,vitór,viv,volt,votuporang,wwwaquinoticiascom,zon,áre,óbit
0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
# obtendo a VSM com TFIDF
def compute_vsm_tfidf(dataset,lang,domain_stopwords=[]):
  
  d = []
  for index,row in dataset.iterrows():
    text = row['title'] #texto do evento
    text2 = remove_stopwords(text, lang,domain_stopwords)
    text3 = stemming(text2, lang)
    d.append(text3)
  
  matrix = TfidfVectorizer()
  X = matrix.fit_transform(d)
  
  tfidf_vect_df = pd.DataFrame(X.todense(), columns=matrix.get_feature_names())

  return tfidf_vect_df


vsm = compute_vsm_tfidf(dataset,'portuguese')
vsm

,2ª,acab,acontec,adot,afet,afirm,aft,al,alago,alarm,alert,algum,almg,amarel,ampl,ano,anunc,apo,apont,apur,após,aqu,aquidau,ar,aracaju,arax,assinat,atend,atenç,aterror,atest,ating,atrás,aument,açõ,bah,barbacen,barr,bh,biólog,...,sob,sobr,solicit,som,sp,sup,surt,suspeit,tabapuã,temp,ter,teófil,to,tod,tragéd,tranquiliz,tribun,tripl,troc,trê,uberab,un,unidad,uol,urban,vacin,vai,val,valad,vej,vint,vir,vitór,viv,volt,votuporang,wwwaquinoticiascom,zon,áre,óbit
0,0.450408,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.117617,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.399391,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.462171,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.233465,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.157503,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.130137,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.450408,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.117617,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.399391,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.462171,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.233465,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.387121,0.0,0.0,0.089434,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.157503,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
6,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.416038,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.36462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
7,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.157503,0.000000

In [20]:
# computando dissimilaridade de cosseno

def dis_cosine(matrix, e1, e2):
  dcos = cosine(matrix.iloc[e1,:], matrix.iloc[e2,:])
  return dcos


# exemplo: dissimilaride entre o primeiro (id=0) e o segundo evento (id=1) do vsm-tfidf
dis_cosine(vsm,0,1)

0.8801264212912872

In [21]:
# calculando a rede de eventos por proximidade de conteudo
def compute_network(matrix,seed=0,min_dcos=0.7,max_neighbors=3,max_nodes=1000):
  
  G=nx.Graph()
  visited = []
  visited.append(seed)
  total = matrix.shape[0]
  temp = {}
  
  while(True):
  
    seed = visited.pop()
    temp[seed]=1
  
    neighbors = {}

    for i in range(0,total):
      if seed==i: continue
      if i in temp: continue
      dcos = dis_cosine(matrix,i,seed)
      if dcos <= min_dcos:
        neighbors[i] = dcos

    sorted_x = sorted(neighbors.items(), key=lambda kv: kv[1])

    counter=0
    for item in sorted_x:
      G.add_edge(seed, item[0], weight=(1-item[1]))
      if item[0] not in temp: visited.append(item[0])
      counter+=1
      if (counter >= max_neighbors): break     

  
    if(len(G) >= max_nodes): break
    if(len(visited)==0): break
    
    
    
  return G
    
G = compute_network(vsm,min_dcos=0.8,seed=1)
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 287\nNumber of edges: 701\nAverage degree:   4.8850'

In [22]:
# visualizando a rede de eventos por proximidade de conteudo
def plot_event_network():
  
  # plotando rede
  pos = nx.drawing.layout.spring_layout(G)
  for item in pos:
    G.node[item]['pos']=[pos[item][0],pos[item][1]]
    
  edge_x = []
  edge_y = []
  for edge in G.edges():
      x0, y0 = G.node[edge[0]]['pos']
      x1, y1 = G.node[edge[1]]['pos']
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=0.5, color='#888'),
      hoverinfo='none',
      mode='lines')

  node_x = []
  node_y = []
  for node in G.nodes():
      x, y = G.node[node]['pos']
      node_x.append(x)
      node_y.append(y)

  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      marker=dict(
          showscale=True,
          # colorscale options
          #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
          #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
          #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
          colorscale='YlGnBu',
          reversescale=True,
          color=[],
          size=10,
          colorbar=dict(
              thickness=15,
              title='Node Connections',
              xanchor='left',
              titleside='right'
          )))
  
  node_adjacencies = []
  node_text = []
  for node, adjacencies in enumerate(G.adjacency()):
      node_adjacencies.append(len(adjacencies[1]))

  for node in G.nodes():
    node_text.append(str(dataset.iloc[node,:]['title']))
      
  node_trace.marker.color = node_adjacencies
  node_trace.text = node_text
  
  fig = go.Figure(data=[edge_trace, node_trace],
               layout=go.Layout(
                  title='Event Network',
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  annotations=[ dict(
                      text="",
                      showarrow=False,
                      xref="paper", yref="paper",
                      x=0.005, y=-0.002 ) ],
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.show()

plot_event_network()